In [1]:
from random import random, sample
import numpy as np

from astropy import coordinates
from astroquery.esa.hubble import ESAHubbleClass
from astropy.io import fits
from astroquery.mast import Observations
from astropy.table import vstack, Table

import matplotlib.pyplot as plt
from PIL import Image

import sys
import os
import time
import re
import json
 
import requests
from urllib.parse import quote as urlencode

In [2]:
def query_hubble(coords: list):

    prod_table = None

    flag = True
    #Verilen parametrelerle 5 arcsec yarıçapında arama
    for i in range(len(coords)):

        table = Observations.query_region(str(coords[i].ra) + " " + str(coords[i].dec), radius = "2.0 deg", pagesize = 10000, page = 1)

        # Filter results to include only HST observations (excluding HLA), optical, image data, and science intent
        filtered_obs = table[
            (table['obs_collection'] == 'HST') &
            (table['dataproduct_type'] == 'image') &  # Image data
            (table['wavelength_region'] == 'OPTICAL') &  # Optical wavelength
            (table['intentType'] == 'science') &  # Science intent
            (table['dataRights'] == 'PUBLIC')  # Exclude HLA data
        ]

        if(len(filtered_obs) != 0):
            temp_products = Observations.get_product_list(filtered_obs)
            temp_products = temp_products[temp_products["productType"] == "SCIENCE"]
            temp_products = temp_products[temp_products["calib_level"] == 3]
            temp_products = temp_products[temp_products["productSubGroupDescription"] == "DRZ"]
            temp_products = temp_products[temp_products["obs_collection"] != "HLA"]
            temp_products = temp_products[temp_products["filters"] != "detection"]
            temp_products = temp_products[temp_products["filters"] != "DETECTION"]

            
            if flag:
                flag = False
                prod_table = temp_products
        
            else:
                prod_table = vstack([prod_table, temp_products])
        
        else:
            print("Gözlem verisi yok!")

        print(i)

        del table

        print(len(prod_table))

    return prod_table